In [1]:
import pandas as pd
import numpy as np
import seaborn as sb

In [2]:
from google.colab import drive

drive.mount('/content/gdrive')
root_path = 'gdrive/My Drive/Mineração'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
path = "/content/gdrive/My Drive/Mineração/"
v = pd.read_csv(path + "V.csv")
fgr = pd.read_csv(path + "FGR.csv", decimal=",")
paradas = pd.read_csv(path + "paradas.csv")
rota_paradas = pd.read_csv(path + "rota_paradas.csv")
rotas = pd.read_csv(path + "rotas.csv")
dados_bairros = pd.read_csv(path + "dados_bairros.csv")

- Demanda
- Tamanho da linha
- Horario
- IDH dos bairros
- É domingo?

In [4]:
demanda_p_linha_dia = v.groupby(["linha_numero", "movimento_diario_data_mov"]).count()["passageiro_matricula"].to_frame().reset_index()
demanda_p_linha_dia.rename(columns={"passageiro_matricula": "demanda_do_dia"}, inplace = True)

In [5]:
demanda_p_linha_dia

,linha_numero,movimento_diario_data_mov,demanda_do_dia
0,1,2021-08-29,232
1,1,2021-08-31,9096
2,1,2021-09-01,8888
3,1,2021-09-02,8244
4,1,2021-09-03,9020
...,...,...,...
2078,942,2021-09-05,2
2079,991,2021-08-31,76
2080,991,2021-09-01,55
2081,991,2021-09-02,55


In [6]:
# Demanda por intervalor de tempo
v["passageiro_data_hora"] = pd.to_datetime(v["passageiro_data_hora"])
v["passageiro_hora_entrada"] = v["passageiro_data_hora"].dt.hour
v["passageiro_hora_range"] = pd.cut(v["passageiro_hora_entrada"], bins=[-1, 4, 6, 9, 12, 15, 18, 21, float('Inf')], labels=['00-3h59', '4-5h59', '6-8h59', '9-11h59', '12-14h59', '15-17h59', '18-20h59', '21-23h59'])
v_hora_range = v.groupby(["linha_numero", "movimento_diario_data_mov", "passageiro_hora_range"]).count()["passageiro_matricula"].to_frame().reset_index()
v_hora_range.dropna(inplace=True)

In [7]:
v_hora_range

,linha_numero,movimento_diario_data_mov,passageiro_hora_range,passageiro_matricula
0,1,2020-11-10,00-3h59,0
1,1,2020-11-10,4-5h59,0
2,1,2020-11-10,6-8h59,0
3,1,2020-11-10,9-11h59,0
4,1,2020-11-10,12-14h59,0
...,...,...,...,...
48179,999,2021-09-07,9-11h59,0
48180,999,2021-09-07,12-14h59,0
48181,999,2021-09-07,15-17h59,0
48182,999,2021-09-07,18-20h59,0


In [8]:
v_hora_range.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48184 entries, 0 to 48183
Data columns (total 4 columns):
 #   Column                     Non-Null Count  Dtype   
---  ------                     --------------  -----   
 0   linha_numero               48184 non-null  int64   
 1   movimento_diario_data_mov  48184 non-null  object  
 2   passageiro_hora_range      48184 non-null  category
 3   passageiro_matricula       48184 non-null  int64   
dtypes: category(1), int64(2), object(1)
memory usage: 1.5+ MB


In [9]:
days_week ={
    0: 'Segunda', 
    1: 'Terça', 
    2: 'Quarta', 
    3: 'Quinta', 
    4: 'Sexta',
    5: 'Sábado', 
    6: 'Domingo'
}

v_hora_range["movimento_diario_data_mov"] = pd.to_datetime(v_hora_range["movimento_diario_data_mov"])

def eh_domingo(dia_semana):
    if dia_semana == "Domingo":
        return 1
    else:
        return 0

v_hora_range["Dia_Semana"] = v_hora_range.movimento_diario_data_mov.dt.weekday
v_hora_range["Dia_Semana"] = v_hora_range.Dia_Semana.map(days_week)
v_hora_range["Domingo"] = v_hora_range["Dia_Semana"].map(eh_domingo)

In [10]:
v_hora_range

,linha_numero,movimento_diario_data_mov,passageiro_hora_range,passageiro_matricula,Dia_Semana,Domingo
0,1,2020-11-10,00-3h59,0,Terça,0
1,1,2020-11-10,4-5h59,0,Terça,0
2,1,2020-11-10,6-8h59,0,Terça,0
3,1,2020-11-10,9-11h59,0,Terça,0
4,1,2020-11-10,12-14h59,0,Terça,0
...,...,...,...,...,...,...
48179,999,2021-09-07,9-11h59,0,Terça,0
48180,999,2021-09-07,12-14h59,0,Terça,0
48181,999,2021-09-07,15-17h59,0,Terça,0
48182,999,2021-09-07,18-20h59,0,Terça,0


In [11]:
# Variável de km da rota
linha_p_km_programado = fgr[["fechamento_data", "linha_numero", "linha_km_programado"]].drop_duplicates()
linha_p_km_programado = linha_p_km_programado[linha_p_km_programado.linha_km_programado != 0]
linha_demanda_km = demanda_p_linha_dia.merge(linha_p_km_programado, how="inner", left_on=["linha_numero", "movimento_diario_data_mov"], right_on=["linha_numero","fechamento_data"])
linha_demanda_km.drop(columns=["fechamento_data"], inplace=True)
linha_demanda_km = linha_demanda_km.groupby(["linha_numero"]).mean()[["linha_km_programado"]].reset_index()
dataset_model = v_hora_range.merge(linha_demanda_km, how="inner", on="linha_numero")

In [12]:
rota_paradas_merged = rota_paradas.merge(paradas, how="inner", on="stop_id")
rota_paradas_merged = rota_paradas_merged.merge(rotas, how="inner", on="route_id")
rotas_paradas_clean = rota_paradas_merged[["route_id", "stop_sequence", "district", "city"]]
rotas_paradas_demanda = rotas_paradas_clean.merge(demanda_p_linha_dia, how="inner", left_on="route_id", right_on="linha_numero")
rotas_paradas_demanda.drop(columns=["route_id", "stop_sequence"], inplace=True)
rotas_paradas_demanda = rotas_paradas_demanda[rotas_paradas_demanda["city"] == "Fortaleza"]

In [13]:
!pip install unidecode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [14]:
# Retira acentos e barras (/ + termo) dos nomes dos bairros
from unidecode import unidecode

district = []
for index, row in rotas_paradas_demanda.iterrows():
  district_without_punt = unidecode(row["district"])
  district_without_barra = district_without_punt.split("/")[0]
  if district_without_barra.startswith("Parque "):
    district_without_barra = district_without_barra.replace("Parque ", "")
  if district_without_barra.startswith("Vila "):
    district_without_barra = district_without_barra.replace("Vila ", "")
  if district_without_barra.startswith("Prefeito "):
    district_without_barra = district_without_barra.replace("Prefeito ", "")
  district.append(district_without_barra.strip())
rotas_paradas_demanda["Bairro"] = district

district = []
for index, row in dados_bairros.iterrows():
  district_without_punt = unidecode(row["Bairros"])
  district_without_barra = district_without_punt.split("/")[0]
  if district_without_barra.startswith("Parque "):
    district_without_barra = district_without_barra.replace("Parque ", "")
  if district_without_barra.startswith("Vila "):
    district_without_barra = district_without_barra.replace("Vila ", "")
  if district_without_barra.startswith("Prefeito "):
    district_without_barra = district_without_barra.replace("Prefeito ", "")
  district.append(district_without_barra.strip())
dados_bairros["Bairro"] = district

In [15]:
rotas_paradas_demanda["Bairro"] = rotas_paradas_demanda["Bairro"].str.lower()
rotas_paradas_demanda["Bairro"] = rotas_paradas_demanda["Bairro"].str.replace("ç", "c")
dados_bairros["Bairro"] = dados_bairros["Bairro"].str.lower()

In [16]:
rotas_demanda_bairro = rotas_paradas_demanda.merge(dados_bairros, how="inner", left_on="Bairro", right_on="Bairro")
rotas_media_idh = rotas_demanda_bairro.groupby(["linha_numero"]).mean()[["IDH em 2010[8]"]]

In [17]:
dataset_model = dataset_model.merge(rotas_media_idh, how="inner", on="linha_numero")

In [18]:
rotas_idh_max = rotas_demanda_bairro.groupby(["linha_numero"]).max()[["IDH em 2010[8]"]].reset_index()

In [19]:
dataset_model = dataset_model.merge(rotas_idh_max, how="inner", on="linha_numero")

GET DUMMIES HORA RANGE

In [20]:
dataset_model = pd.concat([dataset_model, pd.get_dummies(dataset_model["passageiro_hora_range"])], axis = 1)

In [21]:
dataset_model.drop(columns=["Dia_Semana", "movimento_diario_data_mov", "linha_numero", "passageiro_hora_range"], inplace=True)

In [22]:
dataset_model

,passageiro_matricula,Domingo,linha_km_programado,IDH em 2010[8]_x,IDH em 2010[8]_y,00-3h59,4-5h59,6-8h59,9-11h59,12-14h59,15-17h59,18-20h59,21-23h59
0,0,0,1165.120,0.465660,0.762,1,0,0,0,0,0,0,0
1,0,0,1165.120,0.465660,0.762,0,1,0,0,0,0,0,0
2,0,0,1165.120,0.465660,0.762,0,0,1,0,0,0,0,0
3,0,0,1165.120,0.465660,0.762,0,0,0,1,0,0,0,0
4,0,0,1165.120,0.465660,0.762,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
39971,0,0,1157.175,0.291214,0.530,0,0,0,1,0,0,0,0
39972,0,0,1157.175,0.291214,0.530,0,0,0,0,1,0,0,0
39973,0,0,1157.175,0.291214,0.530,0,0,0,0,0,1,0,0
39974,0,0,1157.175,0.291214,0.530,0,0,0,0,0,0,1,0


In [23]:
dataset_model.rename(columns={"passageiro_hora_range": "hora_range", 
                              "passageiro_matricula": "demanda",
                              "Domingo": "domingo", 
                              "linha_km_programado": "linha_km",
                              "IDH em 2010[8]_x": "media_idh",
                              "IDH em 2010[8]_y": "idh_max"}, inplace=True)

In [24]:
dataset_model['demanda'] = dataset_model['demanda'].replace(0, np.nan)

In [25]:
dataset_model.dropna(inplace=True)

# Pre Processamento

In [26]:
X = dataset_model.drop(columns=["demanda"]).values
Y = dataset_model["demanda"].values

In [27]:
from sklearn.preprocessing import StandardScaler
standard = StandardScaler()

X = standard.fit_transform(X)

In [28]:
X

array([[-0.37452726,  0.50449666,  0.78716743, ..., -0.39214153,
        -0.39173309, -0.36174662],
       [-0.37452726,  0.50449666,  0.78716743, ..., -0.39214153,
        -0.39173309, -0.36174662],
       [-0.37452726,  0.50449666,  0.78716743, ..., -0.39214153,
        -0.39173309, -0.36174662],
       ...,
       [-0.37452726,  0.49316187, -0.59505574, ...,  2.55009969,
        -0.39173309, -0.36174662],
       [-0.37452726,  0.49316187, -0.59505574, ..., -0.39214153,
         2.55275856, -0.36174662],
       [-0.37452726,  0.49316187, -0.59505574, ..., -0.39214153,
        -0.39173309,  2.76436587]])

In [29]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import RidgeClassifier
from sklearn.linear_model import ElasticNet 
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [30]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, train_size=0.75, test_size=0.25, random_state=123)

In [31]:
linear_regression = LinearRegression()
linear_regression.fit(X_train, y_train)
results = linear_regression.predict(X_test)

In [32]:
results

array([ 228.73787275, -196.01212725,  -60.38712725, ...,  -26.13712725,
        159.86287275,  384.11287275])

In [33]:
mean_absolute_error(y_test, results)

146.31361827214516

In [34]:
np.sqrt(mean_squared_error(y_test, results))

223.63570100465753

In [35]:
r2_score(y_test, results)

0.4906930644693611

Lasso

In [36]:
lasso = Lasso()
lasso.fit(X_train, y_train)
results = lasso.predict(X_test)

In [37]:
mean_absolute_error(y_test, results)

145.7614595970759

In [38]:
np.sqrt(mean_squared_error(y_test, results))

223.57570468197812

In [39]:
r2_score(y_test, results)

0.49096629852188767

Ridge

In [40]:
ridge_classifier = RidgeClassifier()
ridge_classifier.fit(X_train, y_train)
results = ridge_classifier.predict(X_test)

In [41]:
mean_absolute_error(y_test, results)

177.89127645926877

In [42]:
np.sqrt(mean_squared_error(y_test, results))

332.8118059864717

In [43]:
r2_score(y_test, results)

-0.12796277583758542

ElasticNet

In [44]:
elastic_net = ElasticNet()
elastic_net.fit(X_train, y_train)
results = elastic_net.predict(X_test)

In [45]:
mean_absolute_error(y_test, results)

144.03849382971123

In [46]:
np.sqrt(mean_squared_error(y_test, results))

233.9209566871839

In [47]:
r2_score(y_test, results)

0.4427685974927017

## Redes Neurais

In [48]:
from keras.models import Sequential
from keras.layers import Dense

In [49]:
X.shape

(12470, 12)

In [50]:
regressor = Sequential()
regressor.add(Dense(units=6, activation= 'relu', input_dim=12))
regressor.add(Dense(units=3, activation= 'relu'))
regressor.add(Dense(units=1, activation= 'linear'))
regressor.compile(optimizer= 'adam', loss= 'mean_absolute_error', metrics= ['mean_absolute_error'])
regressor.fit(X_train, y_train, batch_size=2, epochs=20)

Epoch 1/20
4676/4676 [==============================] - 23s 5ms/step - loss: 160.1398 - mean_absolute_error: 160.1398
Epoch 2/20
4676/4676 [==============================] - 16s 3ms/step - loss: 123.2573 - mean_absolute_error: 123.2573
Epoch 3/20
4676/4676 [==============================] - 9s 2ms/step - loss: 118.6014 - mean_absolute_error: 118.6014
Epoch 4/20
4676/4676 [==============================] - 7s 2ms/step - loss: 116.7071 - mean_absolute_error: 116.7071
Epoch 5/20
4676/4676 [==============================] - 7s 2ms/step - loss: 115.4203 - mean_absolute_error: 115.4203
Epoch 6/20
4676/4676 [==============================] - 7s 2ms/step - loss: 114.5271 - mean_absolute_error: 114.5271
Epoch 7/20
4676/4676 [==============================] - 7s 2ms/step - loss: 113.8772 - mean_absolute_error: 113.8772
Epoch 8/20
4676/4676 [==============================] - 7s 2ms/step - loss: 113.2988 - mean_absolute_error: 113.2988
Epoch 9/20
4676/4676 [==============================] - 7s 2ms

In [51]:
results = regressor.predict(X_test)

In [52]:
mean_absolute_error(y_test, results)

104.35240284917903

In [53]:
np.sqrt(mean_squared_error(y_test, results))

200.84370615084921

In [54]:
r2_score(y_test, results)

0.5892157597376866